<a href="https://colab.research.google.com/github/jinsusong/21-study-paper-review/blob/main/BERT_Classification_HuggingFace_Toxic_Comment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### BERT Classification - HuggingFace Toxic Comment

huggingface transformers lib 사용을 위해 설치 

In [ ]:
!pip install transformers==2.3.0

분석 할 데이터 압축 해제

In [ ]:
dataset_directory = 'jigsaw-toxic-comment-classification-challenge'

지정 경로 설정해서 압축 해제 

In [ ]:

!mkdir jigsaw-toxic-comment-classification-challenge
!unzip {dataset_directory} -d jigsaw-toxic-comment-classification-challenge

In [ ]:
!mkdir data
!unzip {dataset_directory}/train.csv.zip -d data/
!unzip {dataset_directory}/test.csv.zip -d data/
!unzip {dataset_directory}/test_labels.csv.zip -d data/
!unzip {dataset_directory}/sample_submission.csv.zip -d data/

In [ ]:
from tqdm.notebook import tqdm # 반복문에서 진행상황 확인 가능 
import numpy as np
import pandas as pd
import tensorflow as tf

### Data Pipeline

- 데이터 셋 로드 
- 데이터 전처리 (Tokenization, Truncation & Padding)
- 데이터 파이프라인 생성 : 데이터를 생성해서 무사히 저장하기까지 일련의 과정

In [ ]:
 train_path = 'data/train.csv'
 test_path = 'data/test.csv'
 test_labels_path = 'data/test_labels.csv'
 subm_path = 'data/sample_submission.csv'


In [ ]:
label_cols = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']

df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
df_test_labels = pd.read_csv(test_labels_path)
df_test_labels = df_test_labels.set_index('id') # 데이터 셋의 index를 id로 변경
df_test_labels.head()




In [ ]:
from transformers import BertTokenizer
from keras.preprocessing.sequence import pad_sequences

bert_model_name = 'bert-base-uncased'

tokenizer = BertTokenizer.from_pretrained(bert_model_name, do_lower_case=True)
# do_lower_case : 토큰화할 때 입력을 소문자로 할지 여부
MAX_LEN = 128

def tokenize_sentences(sentences, tokenizer, max_seq_len = 128):
    tokenized_sentences = []

    for sentence in tqdm(sentences):
        tokenized_sentence = tokenizer.encode(
            sentence, # Sentence to encode
            add_special_tokens  = True , # Add '[CLS]' and '[SEP]'
            max_length = max_seq_len, # Truencate all sentences,
        )
        tokenized_sentences.append(tokenized_sentence)
    
    return tokenized_sentences

def create_attention_masks(tokenized_and_padded_sentences):
    attention_masks = []

    for sentence in tokenized_and_padded_sentences:
        att_mask = [int(token_id > 0) for token_id in sentence]

    return np.asarray(attention_masks)

# 토크나이징 작업
input_ids = tokenize_sentences(df_train['comment_text'], tokenizer, MAX_LEN)
print("토크나이저 후 " , input_ids)

# 패딩 작업 
# maxlen : 정수, 모든 시퀀스의 최대 길이
# dtype : 출력 시퀀스의 자료형. 
# padding: 'pre' 혹은 'post': 각 시퀀스의 처음 혹은 끝을 패딩
# truncating: 'pre' 혹은 'post': maxlen보다 큰 시퀀스의 처음 혹은 끝의 값들을 제거
# value: 부동소수점 혹은 문자열, 패딩할 값.

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
print("패딩 후 : ", input_ids)

# 패딩 된 데이터를 기반으로 어텐션 마스크 생성
attention_masks = create_attention_masks(input_ids)
print("어텐션 마스크 생성 :", attention_masks)


scikit-learn의 패키지 안에 train_test_split 모듈을 활용하여 train set(학습 데이터 셋)과 validation set(테스트 셋)을 분리

In [ ]:
from sklearn.model_selection import train_test_split
# scikit-learn의 패키지 안에 train_test_split 모듈을 활용하여 train set(학습 데이터 셋)과 validation set(테스트 셋)을 분리
# 기존 train / test로 구분 되어 있었던 데이터 셋을 
# train에서 train / validation으로 일정 비율 쪼갠 다음에 학습 시에는 train 셋으로 학습 후 중간중간 validation 셋으로 학습한 모델 평가
# 매 epoch 마다 validation의 오차율을 확인하면서 과적합을 방지해야 좋은 성능의 모델을 만들 수 있다.

labels = df_train[label_cols].values

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=0, test_size=0.1)

train_size = len(train_inputs)
validation_size = len(validation_inputs)


In [ ]:
BATCH_SIZE = 32 
NR_EPOCHES = 1

def create_dataset(data_tuple, epochs=1, batch_size=32, buffer_size=10000, train=True):
    dataset = tf.data.Dataset.from_tensor_slices(data_tuple) # from_tensor_slices : 차원을 맞춰서 데이터를 변환 ex) .slices(([1,2,],[3,4,],[5,6])) -> ((1,3,5),(2,4,6))

    if train: 
        dataset = dataset.shuffle(buffer_size=buffer_size) # shuffle : 버퍼에서 요소를 무작위로 샘플링하여 선택한 요소를 새 요소로 바꾼다.
    
    dataset = dataset.repeat(epochs) # repeat : 데이터 세트를 반복하여 생성 ex)  [1, 2, 3].repeat(3) -> [1,2,3,1,2,3,1,2,3,1,2,3]
    dataset = dataset.batch(batch_size) # batch : 데이터 세트의 요소를 일괄 처리로 결합 ex) (1,2,3,4,5,6,7,8,9).batch_size(3) -> (1,2,3),(4,5,6),(7,8,9)

    if train:
        dataset = dataset.prefetch(1) # prefetch : Dataset이 데이터세트에서 요소를 미리 가져 오는 것 , 

    return dataset 


train_dataset = create_dataset((train_inputs, train_labels), epochs=NR_EPOCHES, batch_size=BATCH_SIZE)
validation_dataset = create_dataset((validation_inputs, validation_labels),epochs=NR_EPOCHES, batch_size=BATCH_SIZE)

#mask 사라짐??? 코드 오류? 
# train_dataset = create_dataset((train_inputs, train_masks, train_labels), epochs=NR_EPOCHS, batch_size=BATCH_SIZE)


### BERT Model 

- Transformers library에서  pretrained BERT base-model을 로드 
- BERT 출력(CLS 토큰에 해당)에서 첫 번째 hidden-state 를 취하여 6개의 뉴런과 시그모이드 활성화(Classifier)가 있는 Dense layer에 공급한다. 
- 이 계층의 출력은 6개의 클래스 각각에 대한 확률로 해석될 수 있다.


In [ ]:
from transformers import TFBertModel
from tensorflow.keras.layers import Dense, Flatten 

class BertClassifier(tf.keras.Model):
    def __init__(self, bert: TFBertModel, num_classes: int):
        super().__init__()
        self.bert = bert 
        self.classifier = Dense(num_classes, activation='sigmoid')

    @tf.function 
    def call(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None):
        outputs = self.bert(input_ids,
                            attention_mask = attention_mask,
                            token_type_ids=token_type_ids,
                            position_ids= position_ids,
                            head_mask=head_mask)
        cls_output = outputs[1]
        cls_output = self.classifier(cls_output)

        return cls_output
model = BertClassifier(TFBertModel.from_pretrained(bert_model_name), len(label_cols))




###Training Loop

- BinaryCrossentropy를 손실 함수로 사용합니다(출력 6개의 각 출력 뉴런에 대해 계산됨...).이는 6개의 이진 분류 작업을 동시에 교육하는 것과 같습니다.)

- Transformers 라이브러리에서 AdamW optimizer를 1-cycle-policy과 함께 사용

- AUC 평가 지표

In [ ]:
import time 
from transformers import create_optimizer

steps_per_epoch = train_size // BATCH_SIZE
validation_steps = validation_size // BATCH_SIZE

# | Loss Function 
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=False)
train_loss = tf.keras.metrics.Mean(name='train_loss') # metrics.Mean : 주어진 값의 (가중된) 평균을 계산, ex) [1, 3, 5, 7]이면 평균은 4. 가중치가 [1, 1, 0, 0]으로 지정된 경우 평균은 2
validation_loss = tf.keras.metrics.Mean(name='test_loss')

# | Optimizer (with 1-cycle-policy)
warmup_steps = steps_per_epoch // 3 
total_steps = steps_per_epoch * NR_EPOCHS - warmup_steps 
optimizer = create_optimizer(init_lr=2e-5, num_train_steps=total_steps, num_warmup_steps=warmup_steps)

# | Metrics
train_auc_metrics = [tf.keras.metrixs.AUC() for i in range(len(label_cols))] # metrixs.AUC() :  리만 합계를 통해 근사 AUC (곡선 아래 영역)를 계산
validation_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]


@tf.function
def train_step(model, token_ids, masks, labels):
    labels = tf.dtypes.cast(labels, tf.float32)

    predictions = model(token_ids, attention_mask=masks, training=False)
    v_loss = loss_object(labels, predictions)

    validation_loss(v_loss)
    for i, auc in enumerate(validation_auc_metrics):
        auc.update_state(labels[:,i], predictions[:,i])

def train(model, train_dataset, val_dataset, train_steps_per_epoch, val_steps_per_epoch, epochs):
    for epoch in range(epochs):
        print('=' * 50, f"EPOCH{epoch}",'='*50)
        start = time.time()

        for i, (token_ids, masks, labels) in enumerate(tqdm(trainn_dataset, total=train_steps_per_epoch)):
            train_step(model, token_ids, masks, labels)
            if i % 1000 == 0:
                print(f'\nTrain Step: {i}, Loss: {train_loss.result()}')
                for i, label_name in enumertate(label_cols):
                    print(f"{label_name} roc_auc {train_auc_metrics[i].result()}")
                    train_auc_metrics[i].reset_states()
        
        for i, (token_ids, masks, labels) in enumerate(tqdm(val_dataset, total=val_steps_per_epoch)):
            validation_step(model, token_ids, masks, labels)
        
        print(f'\nEpoch {epoch+1}, Validation Loss: {validation_loss.result()}, Time: {time.time()-start}\n')

        for i, label_name in enumerate(label_cols):
            print(f"{label_name} roc_auc {validation_auc_metrics[i].result()}")
            validation_auc_metrics[i].reset_states()

        print('\n')

train(model, train_dataset, validation_dataset, train_steps_per_epoch=steps_per_epoch, val_steps_per_epoch=validation_steps, epochs=NR_EPOCHES)









